In [2]:
key="7NvUEUX4tnzOja5KQ99gmUG37DQOV9oelvz1akWAr2Zts9X57djRMwbvfgjQoykp"
secret="X9CWCXNsdypjEU8Q0AQaoaqPrcnaX4wpDe5KVxsAfThkVJJAvufiGJ3tb95QqnQC"

# key api test2
import time
from binance_ft.um_futures import UMFutures
um_futures_client = UMFutures(key=key, secret=secret)
# um_futures_client = UMFutures(key=key, secret=secret, base_url="https://testnet.binancefuture.com")

from binance.spot import Spot
spot_client = Spot(api_key=key, api_secret=secret)
import pandas as pd
import os



/tmp/ipykernel_612660/1899344168.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:

pair = "JASMYUSDT"
pair_spot = pair.replace("USDT", "USDT")

future_info = um_futures_client.exchange_info()['symbols']
kk = [i['filters'] for i in future_info if i['symbol'] == pair][0][1]
print("future_info", kk)

print("spot_info", spot_client.exchange_info(pair_spot)['symbols'][0]['filters'][1])

bid_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])
ask_spot = float(spot_client.book_ticker(pair_spot)['askPrice'])

mark_price = float(spot_client.ticker_price(pair_spot)['price'])
mark_ft = float(um_futures_client.mark_price(pair)['markPrice'])
bid_spot, mark_price, ask_spot, mark_ft


future_info {'filterType': 'LOT_SIZE', 'maxQty': '50000000', 'stepSize': '1', 'minQty': '1'}
spot_info {'filterType': 'LOT_SIZE', 'minQty': '0.10000000', 'maxQty': '92141578.00000000', 'stepSize': '0.10000000'}


(0.011072, 0.011078, 0.011075, 0.01106336)

In [ ]:
# spot_client.cancel_open_orders(symbol=pair_spot)

In [26]:
# future truoc spot sau:
precision_ft = 0
precision_spot =1
usdt = 50
um_futures_client.change_leverage(symbol=pair, leverage=5, recvWindow=6000)

# print("order spot at", price_spot)


now_ft_price = float(um_futures_client.mark_price(pair)['markPrice']) 
price_ft = now_ft_price+0.000003
price_ft = round(price_ft, 6)
time_start = time.strftime("%m-%d %H:%M:%S")
print("order ft at", price_ft)

quantity_ft = round((usdt)/now_ft_price, precision_ft)
open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="LIMIT", quantity=quantity_ft, price = price_ft, timeInForce="GTC")

# open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="MARKET", quantity=quantity_ft)
ft_order_id = open_future["orderId"]
print("ft_order_id =", ft_order_id)
while True:
    res_query_ft = um_futures_client.query_order(symbol=pair, orderId=ft_order_id, recvWindow=6000)
    if res_query_ft['status'] == "FILLED":
        # avg_spot = float(spot_order['price'])
        price_ft = float(res_query_ft['avgPrice'])
        executedQty_future = float(res_query_ft['executedQty'])
        usdt_value_ft = float(res_query_ft['cumQuote'])
        print("price_ft =", price_ft)
        print("usdt_value_ft =",usdt_value_ft)
        print("executedQty_future =", executedQty_future)
        print("type_ft =",res_query_ft['type'])
        print("---------------------")
        price_spot = float(spot_client.ticker_price(pair_spot)['price'])
        print("price_spot expect: ", price_spot)
        quantity_spot = round(usdt_value_ft/price_spot,precision_spot)
        open_spot = spot_client.new_order(symbol=pair_spot, side="BUY", type="LIMIT", quantity=quantity_spot, price=price_spot, timeInForce="GTC")
        order_id_spot = open_spot["orderId"]
        while True:
            res_query_spot = spot_client.get_order(symbol=pair_spot, orderId=order_id_spot, recvWindow=6000)
            if res_query_spot['status'] == "FILLED":
                executedQty_spot = float(res_query_spot['executedQty'])
                usdt_value_spot = float(res_query_spot['cummulativeQuoteQty'])
                price_spot = float(res_query_spot['price'])
                # print("spot:", price_spot, usdt_value_spot, "quantity:", executedQty_spot, res_query_spot['type'])
                print("price_spot =", price_spot)
                print("usdt_value_spot =",usdt_value_spot)
                print("executedQty_spot =", executedQty_spot)
                print("type_spot =",res_query_spot['type'])
                break
            time.sleep(0.1)
        break
    time.sleep(0.1)

order ft at 0.014393
ft_order_id = 2747858775
price_ft = 0.014393
usdt_value_ft = 50.015675
executedQty_future = 3475.0
type_ft = LIMIT
---------------------
price_spot expect:  0.014405
price_spot = 0.014405
usdt_value_spot = 50.0156005
executedQty_spot = 3472.1
type_spot = LIMIT


In [27]:
file_hist = "history.xlsx"
if not os.path.exists(file_hist):
    df = pd.DataFrame(columns=["time_start", "pair", "price_open_spot","usdt_spot","price_open_future","usdt_ft", "price_close_spot", "price_close_ft", "time_order", "time_close_spot", "time_close_ft", "pnl", "pnl_per_input", "thresh", "highest_ft"])
    df.to_excel(file_hist,index_label="stt" )



fee_open = 0.00075 # use busd
fee_close = 0.00075

fee_ft_open = 0.00017
fee_ft_close = 0.00017
all_fee_per = fee_open + fee_close +fee_ft_open+fee_ft_close
fee_spot = (fee_open + fee_close)*usdt_value_spot
fee_future = (fee_ft_open + fee_ft_close) * usdt_value_ft
print("fee spot", fee_spot)
print("fee_future", fee_future)
thresh_profit = 0.0008

while True:
    # curret_ft = float(um_futures_client.mark_price(pair)['markPrice'])
    # curret_spot = float(spot_client.ticker_price(pair_spot)['price'])


    curret_ft = float(um_futures_client.book_ticker(pair)['askPrice'])  # > mark price
    curret_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])  # < mark price

    gap_ft = -(curret_ft - price_ft)/price_ft * 100
    gap_spot = (curret_spot - price_spot)/price_spot * 100  

    if gap_spot + gap_ft > 0.25:
        print(gap_spot + gap_ft)

    pnl_ft = (price_ft - curret_ft)/price_ft * usdt_value_ft
    pnl_spot = (curret_spot-price_spot)/price_spot * usdt_value_spot
    
    # print("pnl ft: " ,pnl_ft, "pnl_spot", pnl_spot)

    # print("pnl:",pnl_spot + pnl_ft - fee_spot - fee_future)
    
    if pnl_spot + pnl_ft - fee_spot - fee_future > usdt_value_spot * thresh_profit :
        # clo\
        #  nbbbvse position
        close_future =  um_futures_client.new_order(symbol=pair, side="BUY", type="LIMIT", quantity=executedQty_future, price = curret_ft, timeInForce="GTC")

        # close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
        close_spot = spot_client.new_order(symbol=pair_spot, side="SELL", type="LIMIT", quantity=executedQty_spot, price=curret_spot, timeInForce="GTC")
        close_future_id=close_future['orderId']
        close_spot_id=close_spot['orderId']

        
        # close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
        # close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)
        time_order_close = time.strftime("%m-%d %H:%M:%S")
        print("***************************** ", time_order_close, " ****************")
        print("curret_ft", curret_ft)
        print("curret_spot", curret_spot)
        time.sleep(1)
        print("take order at:")
        time_done_spot = None
        time_done_ft = None

        while True:
            status_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future_id)
            status_spot_close = spot_client.get_order(symbol=pair_spot, orderId=close_spot_id, recvWindow=6000)
            highest_price_ft_when_close = 0
            if status_spot_close['status'] == "FILLED":
                if time_done_spot is None :
                    time_done_spot = time.strftime("%m-%d %H:%M:%S")
                rate_ft = float(um_futures_client.mark_price(pair)['markPrice'])/curret_ft
                if rate_ft > highest_price_ft_when_close:
                    highest_price_ft_when_close = rate_ft
            if len(status_ft_close) and time_done_ft is None:
                time_done_ft = time.strftime("%m-%d %H:%M:%S")
            if status_spot_close['status'] == "FILLED" and len(status_ft_close):
                price_ft_close_real = float(status_ft_close[0]['price'])
                # price_spot_close_real = float(close_spot['fills'][0]['price'])
                price_spot_close_real = curret_spot
                print("ft:", price_ft_close_real, "--at--", time_done_ft )
                print("spot:", price_spot_close_real,"--at--", time_done_spot )
                print("pnl estimate: ", pnl_spot + pnl_ft - fee_spot - fee_future)
                gap = -(curret_ft - price_ft)/price_ft * 100 + (curret_spot - price_spot)/price_spot * 100
                if gap_spot > 0:
                    print("---> tang", gap,"%")
                else:
                    print("---> giam", gap,"%")
                break
            else:
                time.sleep(0.1)

        break
    time.sleep(0.3)

    # calculate profit
BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])

# outcome_spot = sum([float(i['qty']) * float(i['price']) for i in close_spot['fills']]) #taker
outcome_spot = float(status_spot_close['cummulativeQuoteQty']) #maker

res_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future_id)
res_ft_open = um_futures_client.get_account_trades(symbol=pair, orderId=ft_order_id)

# assert float(res_ft_close["qty"]) == quantity_ft and float(res_ft_open["qty"]) == quantity_ft
# commision_close_spot = sum([float(i['commission']) for i in close_spot['fills']])*BNB_price
commision_close_spot = usdt_value_spot * 0.00075
commision_close_future = sum([float(res['commission']) for res in res_ft_close])*BNB_price
commision_open_future = sum([float(res['commission']) for res in res_ft_open])*BNB_price

pnl_ft =  sum([float(res['realizedPnl']) for res in res_ft_close])
pnl_spot = outcome_spot -  usdt_value_spot
all_profit = pnl_ft + pnl_spot - commision_close_future - commision_close_spot*2 - commision_open_future
print("profit real: ",all_profit)



df = pd.read_excel(file_hist, index_col="stt")
df.loc[len(df)] = [time_start, pair, price_spot, usdt_value_spot, price_ft, usdt_value_ft, price_spot_close_real, price_ft_close_real, time_order_close, time_done_spot, time_done_ft, round(all_profit,4), round(all_profit/usdt_value_spot*100, 4), thresh_profit, (highest_price_ft_when_close-1)*100]

df.to_excel(file_hist,index_label="stt")

    

fee spot 0.07502340075
fee_future 0.017005329500000003
0.7827226553229414
*****************************  02-21 23:32:36  ****************
curret_ft 0.013492
curret_spot 0.013616
take order at:


In [ ]:
# (curret_spot - price_spot)/price_spot +   -(curret_ft - price_ft)/price_ft
# (float(close_spot['fills'][0]['price']) - price_spot)/price_spot +   -(float(um_futures_client.get_account_trades(symbol=pair, orderId=close_future['orderId'])[0]['price']) - price_ft)/price_ft
# close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
# close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)


In [ ]:
# response = um_futures_client.get_account_trades(symbol=pair,orderId=ft_order_id, recvWindow=6000)
# BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])
# float(response[0]['commission']) * BNB_price / usdt_value_ft

In [ ]:
commision_close_future / (0.03931*280)*100